# Travelling Salesman Problem (TSP) on a real geospatial data

### Objective function description

* List of addresses has to be given as an output and then the distance matrix is created using Distance Matrix Google API.
* To use the API, one has to have an API key. There is a possibility to create free account for one year with limited amount of resources.
* In order not to call the API everytime an instance of the TSP class is created, one can save the distance matrix to a .csv file and initiate the instance with this file.
* Distances in the distance matrix are real and they also reflect the current traffic. This matrix doesn't have to be symmetric.
* The "distance" holds for space distance or time distance. That means that in the GeoTSP class there is the distance matrix and also a duration matrix. User can decide in which of these two terms to optimize.

# Example Implementation

You can find it in `src/objfun_geotsp.py`, class `GeoTSP`.

Real-world demonstration follows:

In [1]:
# Import path to source directory (bit of a hack in Jupyter)
import sys
import os
pwd = %pwd
sys.path.append(os.path.join(pwd, os.path.join('..', '..', 'src')))

# Ensure modules are reloaded on any change (very useful when developing code on the fly)
%load_ext autoreload
%autoreload 2

In [2]:
# Import external libraries
import numpy as np

# Import our code
from heur_sg import ShootAndGo
from objfun_geotsp import GeoTSP

In [3]:
# initialization
spots_file = "./spots.csv"
# spots_file = "spots.csv"

spots = []
with open(spots_file) as file:
    for line in file:
        spots.append(line)
        print(line)
        
n = len(spots)
spots = list(map(lambda s: s.strip(), spots))  # remove blank spaces
# The API key has to be saved in the gapikey.txt file
# key = open("./gapikey.txt", "r").read()  # API key

dm_path = "."

Kosmonautů 2844, Mělník

Lhotka u Mělníka 13, Lhotka

Bezručova 2, Mělník

Kpt. Jaroše 10, Mělník

Fibichova 5, Mělník

Ovocná 1540, Mělník

Ve Žlábkách 10, Mělník

Mladoboleslavská 13, Mělník

Okružní 2545/9, Mělník

náměstí Míru 10, Mělník

Horní náměstí 15, Jablonec nad Nisou

náměstí Štefánikovo 1, Staré Město, Liberec

Svojsíkova 2, Staré Město, Liberec

Zhořelecká 3, Staré Město, Liberec

Dožínková 239/6, Růžodol I, Liberec

Na Roli 13, Jablonec nad Nisou

Za Hrází, Mšeno nad Nisou, Jablonec nad Nisou

Kosmonautů 2845, Mělník

Lhotka u Mělníka 106, Lhotka

Bezručova 3, Mělník

Kpt. Jaroše 15, Mělník

Fibichova 7, Mělník

Ovocná 1539, Mělník

Ve Žlábkách 9, Mělník

Mladoboleslavská 20, Mělník

Okružní 2544, Mělník

náměstí Míru 11, Mělník

Horní náměstí 16, Jablonec nad Nisou

náměstí Štefánikovo 2, Staré Město, Liberec

Svojsíkova 3, Staré Město, Liberec

Zhořelecká 4, Staré Město, Liberec

Dožínková 240, Růžodol I, Liberec

Na Roli 12, Jablonec nad Nisou


In [4]:
TSP = GeoTSP(spots, dm_path=dm_path, max_rows=5)

In [5]:
print(TSP.dura_matrix)
print(TSP.dist_matrix)

[[  0. 604. 231. 252. 227.]
 [574.   0. 463. 663. 637.]
 [111. 598.   0. 279. 253.]
 [114. 541. 168.   0. 258.]
 [139. 566. 194.  25.   0.]]
[[   0. 6939. 1618. 1267. 1153.]
 [6433.    0. 5922. 7441. 7327.]
 [ 512. 7343.    0. 1898. 1784.]
 [ 465. 6597. 1277.    0. 2077.]
 [ 579. 6711. 1390.  114.    0.]]


In [6]:
# random point generation
x = TSP.generate_point()
print(x)

[3 0 0 0]


In [7]:
# decode this solution (into list of visited cities)
cx = TSP.decode(x)
print(cx)

[0 4 1 2 3]


In [8]:
# what is the cost of such tour?
print(TSP.evaluate(x))
print(TSP.evaluate(x, mtype="duration"))

15684.0
1535.0


In [9]:
# print the route
print(TSP.get_route(x))

Kosmonautů 2844, Mělník -> 
Fibichova 5, Mělník -> 
Lhotka u Mělníka 13, Lhotka -> 
Bezručova 2, Mělník -> 
Kpt. Jaroše 10, Mělník


In [10]:
# get the nearest locations from the one selected
n_loc = 0  # Order number of the location in the 'spots'
k = 1  # Number of nearest locations to be found
print(TSP.find_k_nn(n_loc, k))

[4]


In [11]:
# neighbourhood of x:
N = TSP.get_neighborhood(x, 1)
print(N)

[array([2, 0, 0, 0]), array([3, 1, 0, 0]), array([3, 0, 1, 0])]


In [12]:
# decoded neighbours and their objective function values
for xn in N:
    print('{} ({}) -> {:.4f}'.format(xn, TSP.decode(xn), TSP.evaluate(xn)))

[2 0 0 0] ([0 3 1 2 4]) -> 15570.0000
[3 1 0 0] ([0 4 2 1 3]) -> 17327.0000
[3 0 1 0] ([0 4 1 3 2]) -> 16582.0000


### TSP optimization using Random Shooting ($\mathrm{SG}_{0}$)

In [13]:
heur = ShootAndGo(TSP, maxeval=1000, hmax=0)
print(heur.search())

{'best_y': 9887.0, 'best_x': array([3, 2, 1, 0]), 'neval': inf, 'log_data': Empty DataFrame
Columns: []
Index: []}


### TSP optimization using FSA

In [14]:
from heur_fsa import FastSimulatedAnnealing
from heur_aux import Correction, CauchyMutation
from tqdm import tqdm
import pandas as pd

def experiment_fsa(of, maxeval, num_runs, T0, n0, alpha, r):
    results = []
    for i in tqdm(range(num_runs), 'Testing T0={}, n0={}, alpha={}, r={}'.format(T0, n0, alpha, r)):
        mut = CauchyMutation(r=r, correction=Correction(of))
        result = FastSimulatedAnnealing(of, maxeval=maxeval, T0=T0, n0=n0, alpha=alpha, mutation=mut).search()
        result['run'] = i
        result['heur'] = 'FSA_{}_{}_{}_{}'.format(T0, n0, alpha, r) # name of the heuristic
        result['T0'] = T0
        result['n0'] = n0
        result['alpha'] = alpha
        result['r'] = r
        results.append(result)
    
    return pd.DataFrame(results, columns=['heur', 'run', 'T0', 'n0', 'alpha', 'r', 'best_x', 'best_y', 'neval'])

In [15]:
table_fsa = pd.DataFrame()
maxeval = 100
NUM_RUNS = 10

for T0 in [1e-10, 1e-2, 1, np.inf]:
    res = experiment_fsa(of=TSP, maxeval=maxeval, num_runs=NUM_RUNS, T0=T0, n0=1, alpha=2, r=0.5)
    table_fsa = pd.concat([table_fsa, res], axis=0)

Testing T0=inf, n0=1, alpha=2, r=0.5: 100%|██████████| 10/10 [00:00<00:00, 68.95it/s]


In [16]:
def rel(x):
    return len([n for n in x if n < np.inf])/len(x)

def mne(x):
    return np.mean([n for n in x if n < np.inf])

def feo(x):
    return mne(x)/rel(x)

stats_fsa = table_fsa.pivot_table(
    index=['heur', 'T0'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_fsa = stats_fsa.reset_index()
stats_fsa.sort_values(by=['T0'])

/Users/ondrej.panek/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/ondrej.panek/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,heur,T0,rel
2,FSA_1e-10_1_2_0.5,1.000000e-10,0.0
0,FSA_0.01_1_2_0.5,1.000000e-02,0.0
1,FSA_1_1_2_0.5,1.000000e+00,0.0
3,FSA_inf_1_2_0.5,inf,0.0


In [17]:
heur = FastSimulatedAnnealing(TSP, maxeval=1000, T0=1, n0=1, alpha=2, 
                              mutation=CauchyMutation(r=0.5, correction=Correction(TSP)))
result = heur.search()
print('neval = {}'.format(result['neval']))
print('best_x = {}'.format(result['best_x']))
print('best_y = {}'.format(result['best_y']))

neval = inf
best_x = [3 2 1 0]
best_y = 9887.0


In [18]:
log_data = result['log_data'].copy()
log_data = log_data[['step', 'x', 'f_x', 'y', 'f_y', 'T', 'swap']]  # column re-ordering, for better readability
log_data.head(10)

,step,x,f_x,y,f_y,T,swap
0,0,"[3, 2, 0, 0]",13786.0,"[1, 2, 0, 0]",17554.0,1.000000,False
1,1,"[3, 2, 0, 0]",13786.0,"[3, 2, 0, 0]",13786.0,0.500000,False
2,2,"[3, 2, 0, 0]",13786.0,"[3, 2, 0, 0]",13786.0,0.200000,False
3,3,"[3, 2, 0, 0]",13786.0,"[2, 2, 1, 0]",12077.0,0.100000,True
4,4,"[2, 2, 1, 0]",12077.0,"[1, 2, 0, 0]",17554.0,0.058824,False
5,5,"[2, 2, 1, 0]",12077.0,"[3, 2, 1, 0]",9887.0,0.038462,True
6,6,"[3, 2, 1, 0]",9887.0,"[3, 2, 1, 0]",9887.0,0.027027,False
7,7,"[3, 2, 1, 0]",9887.0,"[3, 0, 1, 0]",16582.0,0.020000,False
8,8,"[3, 2, 1, 0]",9887.0,"[3, 1, 1, 0]",11038.0,0.015385,False
9,9,"[3, 2, 1, 0]",9887.0,"[0, 1, 1, 0]",17847.0,0.012195,False
